# Scrape Data from Lazada with beautifulsoup and selenium
- shop, name, rating, date, review_text

!pip install selenium
!pip install bs4
!pip install pandas

# Code ที่ต้อง Import

In [21]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import os  
from datetime import datetime 
import bs4
import time
import re
import random

In [48]:
#กำหนดตัวแปร
#link ที่ต้องการดึงข้อมูล
link = 'https://www.lazada.co.th/products/40-biore-uv-perfect-milk-spf50-pa-40ml-i228864767-s349752616.html?c=&channelLpJumpArgs=&clickTrackInfo=query%3ABiore%2BUV%2BPerfect%2BMilk%3Bnid%3A228864767%3Bsrc%3ALazadaMainSrp%3Brn%3Aa70469eeccfc5be1b1decd1b165901cc%3Bregion%3Ath%3Bsku%3A228864767_TH%3Bprice%3A229%3Bclient%3Adesktop%3Bsupplier_id%3A100148596%3Bbiz_source%3Ah5_internal%3Bslot%3A5%3Butlog_bucket_id%3A470687%3Basc_category_id%3A5885%3Bitem_id%3A228864767%3Bsku_id%3A349752616%3Bshop_id%3A174773%3BtemplateInfo%3A107882_C_D_E%231103_B_L%23-1_A3%23&freeshipping=1&fs_ab=2&fuse_fs=&lang=en&location=Samut%20Prakan&price=229&priceCompare=skuId%3A349752616%3Bsource%3Alazada-search-voucher%3Bsn%3Aa70469eeccfc5be1b1decd1b165901cc%3BunionTrace%3A2ff6089817278003637818185e%3BoriginPrice%3A22900%3BvoucherPrice%3A22900%3BdisplayPrice%3A22900%3BsinglePromotionId%3A900000037044063%3BsingleToolCode%3ApromPrice%3BvoucherPricePlugin%3A1%3BbuyerId%3A0%3Btimestamp%3A1727800364102&ratingscore=4.940058479532164&request_id=a70469eeccfc5be1b1decd1b165901cc&review=442&sale=4363&search=1&source=search&spm=a2o4m.searchlist.list.5&stock=1'
#ชื่อ file output ใส่ชื่อผลิตภัณฑ์
output = 'Biore UV Perfect Milk'
# Set up Chrome binary_location
location = 'C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe'
#the service path to ChromeDriver
service_path = r'D:\Git\Text-Sentiment-Analysis\chromedriver.exe'

In [49]:
# Set up Chrome options
options = Options()
options.binary_location = location

# Specify the path to ChromeDriver
service = Service(service_path)

driver = webdriver.Chrome(service=service, options=options)

# Open lazada website
driver.get(link)

In [50]:
#Zoom out เพื่อให้ load ทั่วหน้าจอ
driver.execute_script("document.body.style.zoom='10%'")

In [30]:
records=[]

In [ ]:
# การ Scrape ข้อมูลจากหลายหน้าของ Web Shopee
for i in range(7):
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    el=soup.select("#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div")
    # Define star rating image sources
    src_plus_one = "//img.lazcdn.com/g/tps/tfs/TB19ZvEgfDH8KJjy1XcXXcpdXXa-64-64.png"
    src_plus_zero = "//img.lazcdn.com/g/tps/tfs/TB18ZvEgfDH8KJjy1XcXXcpdXXa-64-64.png"

    # Extract shop 
    shop_element = soup.select_one("#module_seller_info > div > div.seller-name-retail > div.seller-name__wrapper > div.seller-name__detail > a.pdp-link.pdp-link_size_l.pdp-link_theme_black.seller-name__detail-name")
    shop = shop_element.text.strip() if shop_element else ""

    for e in el:

        # Extract review text
        review = e.select_one("#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div > div.item-content > div.content")
        review_text = review.text.strip() if review else ""

        # Extract and calculate rating
        stars = e.select_one('#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div > div.top > div')
        rating = 0
        if stars:
            for star in stars.find_all('img'):
                if star['src'] == src_plus_one:
                    rating += 1
                elif star['src'] == src_plus_zero:
                    rating += 0

        # Extract review date
        date_element = e.select_one('#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div > div.top > span')
        date = date_element.text.strip() if date_element else ""

        # Extract reviewer name
        name_element = e.select_one('#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div > div.middle > span:nth-child(1)')
        name = name_element.text.strip() if name_element else ""

        # Append the extracted data to records
        records.append([shop, name, rating, date, review_text])

    wait = WebDriverWait(driver, 20)
    next_button = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[4]/div/div[10]/div[1]/div[2]/div/div/div/div[3]/div[2]/div/button[2]')))
    next_button.click()
    time.sleep(random.uniform(10, 20))
    print(i) #print loop


In [ ]:
records

In [ ]:
len(records)

In [54]:
#เลือก filter_sort review1
wait = WebDriverWait(driver, 20)
filterStar = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[4]/div/div[10]/div[1]/div[2]/div/div/div/div[2]/div/div[2]')))
filterStar.click()

In [ ]:
#เลือก sort เป็นดาวน้อยไปดาวมาก
selectStar = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[9]/div/div/ul/li[4]')))
selectStar.click()

In [58]:
# สร้าง DataFrame และเพิ่มลำดับแถว
df = pd.DataFrame(records, columns=['shop', 'name', 'rating', 'date', 'review_text'])

# เพิ่มคอลัมน์ลำดับแถว (Row Index) โดยใช้ range
df['no'] = range(1, len(df) + 1)

# แสดง DataFrame ที่มีลำดับแถว
df = df[['no', 'shop', 'name', 'rating', 'date', 'review_text']]


In [59]:
# รับเวลาปัจจุบัน  
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")

In [ ]:
if not os.path.exists('RawDataCSV'):
    os.makedirs('RawDataCSV')
if not os.path.exists('RawDataExel'):
    os.makedirs('RawDataExel')

In [60]:
# บันทึก DataFrame ลงไฟล์ CSV
csv_file = f'RawDataCSV/{output}_{current_time}.csv'
df.to_csv(csv_file, index=False)

# บันทึก DataFrame ลงไฟล์ Excel
excel_file = f'RawDataExel/{output}_{current_time}.xlsx'
df.to_excel(excel_file, index=False)